In [1]:
import json
import os

from dotenv import load_dotenv
from numpy.distutils.system_info import openblas64__info
from supabase import create_client, Client
import pandas as pd

from backend.scrapping.courses_epfl import courses

load_dotenv()

URL = os.getenv("SUPABASE_URL")
KEY = os.getenv("SUPABASE_KEY")

url: str = URL
key: str = KEY
supabase: Client = create_client(url, key)

In [23]:
response = (
    supabase.table("courses")
    .select("id", "code")
    .execute()
)

In [24]:
course_codes = set()
for course in response.data:
    course_codes.add(course['code'])

In [5]:
course_codes

{'COM-401',
 'COM-402',
 'COM-407',
 'CS-401',
 'CS-433',
 'CS-438',
 'CS-440',
 'CS-442',
 'CS-450',
 'CS-451',
 'CS-452',
 'CS-460'}

In [30]:
df = pd.read_csv('data/courses.csv')
for index, row in df.iterrows():
    if row['code'] not in course_codes:
        try:
            code = row['code']
            name = row['name'] if pd.notna(row['name']) else None
            ects = row['ects']
            link = row['link']
            professor = row['professor'] if pd.notna(row['professor']) else None
            description = row['description'] if pd.notna(row['description']) else None
            response = (
                supabase.table("courses")
                .insert({"code": code, "name": name, "ects": ects, "link": link, "professor": professor, "description": description})
                .execute()
            )
        except:
            import traceback
            traceback.print_exc()

Traceback (most recent call last):
  File "C:\Users\balanton\AppData\Local\Temp\ipykernel_29156\3266212171.py", line 14, in <module>
    .execute()
     ^^^^^^^^^
  File "D:\Projects\EPFL\AIPM\chillordrill\venv\Lib\site-packages\postgrest\_sync\request_builder.py", line 78, in execute
    raise APIError(r.json())
postgrest.exceptions.APIError: {'code': '23505', 'details': 'Key (code)=(AR-301(a)) already exists.', 'hint': None, 'message': 'duplicate key value violates unique constraint "courses_code_key"'}
Traceback (most recent call last):
  File "C:\Users\balanton\AppData\Local\Temp\ipykernel_29156\3266212171.py", line 14, in <module>
    .execute()
     ^^^^^^^^^
  File "D:\Projects\EPFL\AIPM\chillordrill\venv\Lib\site-packages\postgrest\_sync\request_builder.py", line 78, in execute
    raise APIError(r.json())
postgrest.exceptions.APIError: {'code': '23505', 'details': 'Key (code)=(AR-301(h)) already exists.', 'hint': None, 'message': 'duplicate key value violates unique constraint

In [6]:
df = pd.read_csv('data/courses_comments.csv')
data = df.groupby('course_code')['comment'].count().sort_values(ascending=False).head(10)
most_rated_courses_comments = df[df['course_code'].isin(data.index)]

,code,name,ects,link,professor,description
0,MATH-498,Mathematics projects (master),10,https://edu.epfl.ch/studyplan/en/master/applie...,Profs divers *,L'étudiant applique les compétences acquises d...
1,MATH-416,Abstract analysis on groups,5,https://edu.epfl.ch/studyplan/en/master/applie...,Monod Nicolas,"We study analytic phenomena on groups, notably..."
2,MATH-521,Advanced analytic number theory,5,https://edu.epfl.ch/studyplan/en/master/applie...,NaN,We will present the work of James Maynard (MF ...
3,MATH-535,Algebraic geometry III - selected topics,5,https://edu.epfl.ch/studyplan/en/master/applie...,Schlegel Mejia Sebastian,This course is an introduction to the theory o...
4,MATH-510,Algebraic geometry II - schemes and sheaves,10,https://edu.epfl.ch/studyplan/en/master/applie...,"Filipazzi Stefano, Mukhopadhyay Alapan",The aim of this course is to learn the basics ...
...,...,...,...,...,...,...
1452,PENS-315,SKIL Student Kreativity and Innovation Laboratory,4,https://edu.epfl.ch/studyplan/en/bachelor/proj...,"Cotture Samuel Denis Marcel, Meibom Anders, Tr...",This course will allow students to engage in h...
1453,PENS-308,TRC LC3 Research Platform Fribourg,4,https://edu.epfl.ch/studyplan/en/bachelor/proj...,"Baur Raffael, Corres Sojo Enrique, Fernandez-O...",The UE TRC-LC3 Research Platform Fribourg will...
1454,PENS-309,"Urban neighborhoods, infrastructures and susta...",4,https://edu.epfl.ch/studyplan/en/bachelor/proj...,"Coccolo Silvia, Jessel Beate, Lepesant Hugues,...",Cette Unité d'Enseignement ENAC vise à présent...
1455,PENS-307,Urban planning in the South,4,https://edu.epfl.ch/studyplan/en/bachelor/proj...,"Chenal Jérôme, de Roulet Pablo Txomin Harpo, M...",Ce cours est une introduction aux outils techn...


In [22]:
most_rated_courses_comments

,course_code,course_link,course_name,overall,prof_rating,difficulty,workload,professors,comment
197,COM-401,https://edu.epfl.ch/studyplan/en/master/commun...,Cryptography and security,3.4 (9),3.7,3.6,3.6 (8),['Vaudenay'],"intéressant, bon prof, beaucoup de boulot, fin..."
198,COM-401,https://edu.epfl.ch/studyplan/en/master/commun...,Cryptography and security,3.4 (9),3.7,3.6,3.6 (8),['Vaudenay'],"ils ont un grading system assez spécial, genre..."
199,COM-401,https://edu.epfl.ch/studyplan/en/master/commun...,Cryptography and security,3.4 (9),3.7,3.6,3.6 (8),['Vaudenay'],If you want a challenging but interesting cour...
200,COM-401,https://edu.epfl.ch/studyplan/en/master/commun...,Cryptography and security,3.4 (9),3.7,3.6,3.6 (8),['Vaudenay'],"If you like cryptography, it’s a must do. Howe..."
201,COM-401,https://edu.epfl.ch/studyplan/en/master/commun...,Cryptography and security,3.4 (9),3.7,3.6,3.6 (8),['Vaudenay'],If you are interested by the math behind crypt...
...,...,...,...,...,...,...,...,...,...
1149,MICRO-511,https://edu.epfl.ch/studyplan/en/master/commun...,Image processing I,3.7 (8),4.0,2.1,2.6 (3),"['Unser', 'Van De Ville']",yes if you're lacking the basic concepts of im...
1150,MICRO-511,https://edu.epfl.ch/studyplan/en/master/commun...,Image processing I,3.7 (8),4.0,2.1,2.6 (3),"['Unser', 'Van De Ville']",Basics of image processing. 3 credits pretty e...
1151,MICRO-511,https://edu.epfl.ch/studyplan/en/master/commun...,Image processing I,3.7 (8),4.0,2.1,2.6 (3),"['Unser', 'Van De Ville']","Easy class, especially if you already attended..."
1152,MICRO-511,https://edu.epfl.ch/studyplan/en/master/commun...,Image processing I,3.7 (8),4.0,2.1,2.6 (3),"['Unser', 'Van De Ville']",Easy course but exam a bit long. It's only the...


In [23]:
for index, row in most_rated_courses_comments.iterrows():
    course_code = row['course_code']
    id = dict_.get(course_code)
    if id is not None:
        response = (
            supabase.table("posts")
            .insert({"course_id": id, "content": row["comment"], "is_anonymous": True})
            .execute()
        )